**ส่วนนี้คือการ load ข้อมูล และการกำจัด outliers (ท่านสามารถดูเพิ่มเติมได้ที่ 1_Data_Cleaning.ipynb)**

**ส่วนนี้ได้รวมการสร้าง dummy variables และการทำ data scaling (standardisation) ไว้ด้วย (ท่านสามารถดูเพิ่มเติมได้ที่ 3_Logit.ipynb)**

**สำหรับ Support Vector Machine (SVM) จะเริ่มต่อท้ายหลังจบส่วนนี้**

In [1]:
import sys
sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# disable warnings that are not critical for a clean looking
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import joblib

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
df = pd.read_csv('./dataset/Churn_Modelling.csv')

In [5]:
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)

#### การกำจัด outliers

In [6]:
def my_func(feature, upper, lower):
    tmp = df[(df[feature] >= upper) | (df[feature] <= lower)]
    df.drop(index=tmp.index, inplace=True)

In [7]:
my_func('CreditScore',None,383)
my_func('Age',62,None)

#### การสร้าง dummy variables

In [8]:
df_new = pd.get_dummies(df, drop_first=True)

#### ทำการแบ่งข้อมูล train/test ด้วยอัตราส่วน 80:20

In [9]:
X = df_new.drop('Exited', axis=1)
y = df_new['Exited']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

#### Data Scaling (Standardisation)

In [11]:
from sklearn.preprocessing import StandardScaler

sc_X_train = StandardScaler()
sc_X_test = StandardScaler()

In [12]:
tmp = X_train
std = sc_X_train.fit_transform(tmp)
X_train = pd.DataFrame(std, columns=tmp.columns)

tmp = X_test
std = sc_X_test.fit_transform(tmp)
X_test = pd.DataFrame(std, columns=tmp.columns)

#### เขียนฟังก์ชั่นสำหรับประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [14]:
acc = {}; f1 = {}; prec = {}; recl = {}

def my_eval(model,y_test,predicted):
    
    print(model,'Results')
    print('Accuracy: ', accuracy_score(y_test, predicted))
    print('F1 Score: ', f1_score(y_test, predicted))
    print('Precision: ', precision_score(y_test, predicted))
    print('Recall: ', recall_score(y_test, predicted))
    
    acc[model] = accuracy_score(y_test, predicted)
    f1[model] = f1_score(y_test, predicted)
    prec[model] = precision_score(y_test, predicted)
    recl[model] = recall_score(y_test, predicted)
    
    # declair as a global model to ratain the evalutation values along the notebook
    global df_model_compare
    df_model_compare = pd.DataFrame.from_dict(
        [acc,f1,prec,recl]
    ).rename(
        index={0:'Accuracy',1:'F1 Score',2:'Precision',3:'Recall'}
    )
    #display(df_model_compare)

**Support Vector Machine (SVM) จะเริ่มต้นขึ้นต่อจากนี้**

---

# Support Vector Machine (SVM)

SVM จะสร้างระนาบ (hyperplane) โดยพยายามให้จุดข้อมูลฝั่งลบ (negtive points) อยู่ใต้ระนาบฝั่งลบ (negative hyperplane) และให้จุดข้อมูลฝั่งบวก (positive points) อยู่เหนือระนาบฝั่งบวก (positive hyperplane) และพยายามให้ระนาบทั้งสองห่างกันให้มากที่สุด

โดยใช้การดึงจุดข้อมูลให้อยู่ในมิติที่สูงขึ้น (kernel trick) แล้วใช้ระนาบในการแยกข้อมูลออกจากกันได้

In [15]:
from sklearn.svm import SVC

## การหาค่าที่เหมาะสมที่สุดด้วย Hyperparameter Tuning

In [16]:
from sklearn.model_selection import GridSearchCV

`C` คือ penalty parameter เป็นการ trade off ระหว่างความ smooth ของ decision boundary กับการ classify ได้ถูกต้อง ยิ่งสูงยิ่ง penalty มากยิ่งทำให้ decision boundary แคบ ถ้าสูงมากไปจะกลายเป็น overfitting  

`gamma` ที่สูงขึ้น hyperplanes จะยิ่งพยายาม fit ให้ตรงกับ training point เป๊ะ ๆ ถ้าสูงมากไปจะกลายเป็น overfitting  

`kernel` ใช้ค่า default คือ RBF (Radial Basis Function) เพราะ RBF เป็นฟังก์ชั่นที่ช่วยให้ hyperplane เป็นเส้นโค้งได้ สามารถจัดการข้อมูลตามโลกความจริงซึ่งเป็นแบบ non-linear ได้ 

In [17]:
parameters = {
    'C':[0.01,0.1,1,10,100,1000,10000,25000,50000],
    'gamma':[0.0001,0.001,0.01,0.05,0.1,0.5,1,10],
}

In [18]:
grid_search = GridSearchCV(SVC(random_state=55, kernel='rbf'),
                           parameters,
                           verbose=10,
                           n_jobs=-1,
                           scoring='f1')

แสดง timestamp ก่อนรันอัลกอริทึม:

In [19]:
import datetime

timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

14:17:51.191040


สร้าง (ฝึกสอน) model:

In [20]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=SVC(random_state=55), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000, 10000, 25000,
                               50000],
                         'gamma': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 10]},
             scoring='f1', verbose=3)

แสดง timestamp หลังรันอัลกอริทึม:

In [21]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

14:44:03.152303


In [22]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 0:26:11.961263


**ค่าที่เหมาะสมที่สุด (optimised parameters):**

In [23]:
grid_search.best_params_

{'C': 10000, 'gamma': 0.01}

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [24]:
# Save Model
joblib.dump(grid_search, './models/SVM.pkl')

# Load Model
#grid_search = joblib.load('./models/SVM.pkl')

['./models/SVM.pkl']

## การประเมิน model ที่ใช้ทำนาย

ใช้โมเดลทำนาย test set:

In [25]:
svm_grid_predicted = grid_search.predict(X_test)

In [26]:
confusion_matrix(y_test, svm_grid_predicted)

array([[1465,   41],
       [ 228,  181]], dtype=int64)

[ TN FP ]  
[ FN TP ]

TP (ทำนายว่า churn; ความจริงคือ churn)  
TN (ทำนายว่า non-churn; ความจริงคือ non-churn)  
FP (ทำนายว่า churn; ความจริงคือ non-churn)  
FN (ทำนายว่า non-churn; ความจริงคือ churn)

Churn = เลิกใช้ธนาคาร  
Non-churn = ไม่เลิกใช้ธนาคาร

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [27]:
my_eval('SVM', y_test, svm_grid_predicted)

SVM Results
Accuracy:  0.8595300261096606
F1 Score:  0.5736925515055468
Precision:  0.8153153153153153
Recall:  0.44254278728606355


---

บันทึกค่าประเมินประสิทธิภาพของแต่ละ model เพื่อใช้เปรียบเทียบใน notebook ชื่อ 9_Comparison.ipynb:

In [28]:
df_model_compare.to_csv('./eval_results/eval_results_SVM.csv')